In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def toUpperNoSpaces(row,column):
    return str(row[column]).replace(" ","").upper().strip()
def BoxOfficeNumber(row,column):
    return float(str(row[column]).replace("$","").replace(",","").strip())
data_folder = "C:/Users/Deepika Chandrababu/Desktop/Infoviz/dataset/"

dfAll = pd.read_csv(data_folder + "netflix-rotten-tomatoes-metacritic-imdb.csv")
# dfOM = pd.read_csv('netflixOM.csv')
# dfOS = pd.read_csv('netflixOS.csv')
dfOS = pd.read_csv(data_folder + "NetflixOriginals.csv")
dfOM = pd.read_csv(data_folder + "netflix_original_movie_data.csv")

# Adding new columns for operations like: compare between datasets
dfOM['TitleToCompare'] = dfOM.apply(lambda row: toUpperNoSpaces(row,'Title'), axis = 1)
dfOS['TitleToCompare'] = dfOS.apply(lambda row: toUpperNoSpaces(row,'Title'), axis = 1)
dfAll['TitleToCompare'] = dfAll.apply(lambda row: toUpperNoSpaces(row,'Title'), axis = 1)

dfAll['GenreToCompare'] = dfAll.apply(lambda row: toUpperNoSpaces(row,'Genre'), axis = 1)
dfAll['BoxOfficeProfits'] = dfAll.apply(lambda row: BoxOfficeNumber(row,'Boxoffice'), axis = 1)

# add new column: Original, 1: is produced by netflix, 0: is not produced by netflix
df = dfAll.assign(Original = dfAll.TitleToCompare.isin(pd.concat([dfOM.TitleToCompare,dfOS.TitleToCompare])).astype(int))
df['Year'] = pd.to_numeric(df['Release Date'].str[-4:])
df

,Title,Genre,Tags,Languages,Series or Movie,Hidden Gem Score,Country Availability,Runtime,Director,Writer,...,IMDb Votes,Image,Poster,TMDb Trailer,Trailer Site,TitleToCompare,GenreToCompare,BoxOfficeProfits,Original,Year
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,Thailand,< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,NaN,NaN,LETSFIGHTGHOST,"CRIME,DRAMA,FANTASY,HORROR,ROMANCE",2122065.0,0,2008.0
1,HOW TO BUILD A GIRL,Comedy,"Dramas,Comedies,Films Based on Books,British",English,Movie,7.0,Canada,1-2 hour,Coky Giedroyc,Caitlin Moran,...,2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ,YouTube,HOWTOBUILDAGIRL,COMEDY,70632.0,0,2020.0
2,Centigrade,"Drama, Thriller",Thrillers,English,Movie,6.4,Canada,1-2 hour,Brendan Walsh,"Brendan Walsh, Daley Nixon",...,1720.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BODM2MD...,https://www.youtube.com/watch?v=0RvV7TNUlkQ,YouTube,CENTIGRADE,"DRAMA,THRILLER",16263.0,0,2020.0
3,ANNE+,Drama,"TV Dramas,Romantic TV Dramas,Dutch TV Shows",Turkish,Series,7.7,"Belgium,Netherlands",< 30 minutes,NaN,NaN,...,1147.0,https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BNWRkMz...,NaN,NaN,ANNE+,DRAMA,NaN,0,2016.0
4,Moxie,"Animation, Short, Drama","Social Issue Dramas,Teen Movies,Dramas,Comedie...",English,Movie,8.1,"Lithuania,Poland,France,Iceland,Italy,Spain,Gr...",1-2 hour,Stephen Irwin,NaN,...,63.0,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BODYyNW...,NaN,NaN,MOXIE,"ANIMATION,SHORT,DRAMA",NaN,1,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15475,K-POP Extreme Survival,NaN,"TV Dramas,TV Programmes,TV Comedies,Romantic T...",NaN,Series,NaN,"South Korea,Argentina,United Kingdom,Australia...",< 30 minutes,NaN,NaN,...,NaN,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN,K-POPEXTREMESURVIVAL,NAN,NaN,0,NaN
15476,DreamWorks Shreks Swamp Stories,NaN,"Animal Tales,Family Comedies,Family Adventures...",NaN,Series,NaN,"Russia,Hong Kong,Hungary,Australia,South Korea...",< 30 minutes,NaN,NaN,...,NaN,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN,DREAMWORKSSHREKSSWAMPSTORIES,NAN,NaN,0,NaN
15477,DreamWorks Happy Holidays from Madagascar,"Animation, Comedy, Family","TV Comedies,Kids TV,Animal Tales,TV Cartoons,T...",English,Series,8.4,"Belgium,Switzerland,United States,Germany,Unit...",< 30 minutes,NaN,NaN,...,71.0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,NaN,NaN,NaN,DREAMWORKSHAPPYHOLIDAYSFROMMADAGASCAR,"ANIMATION,COMEDY,FAMILY",NaN,0,NaN
15478,DreamWorks Holiday Classics,"Animation, Comedy, Family","TV Comedies,Kids TV,TV Cartoons,TV Programmes,...",English,Series,8.2,"Belgium,Switzerland,Germany,Argentina,United K...",< 30 minutes,NaN,NaN,...,82.0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN,DREAMWORKSHOLIDAYCLASSICS,"ANIMATION,COMEDY,FAMILY",NaN,0,NaN


In [30]:
#GRAPH 1: TREND OF "--GENRE--" MOVIES/SERIES FROM NETFLIX

dfTrend = pd.DataFrame()
# get unique values from Genre
dfTemp = df['GenreToCompare'].drop_duplicates().str.split(',', expand=True)
dfGenre = dfTemp[0].drop_duplicates()
for col in dfTemp.columns:
    dfGenre = dfGenre.append(dfTemp[col].drop_duplicates(), ignore_index=True)

# drop duplicates and "Nan" value from Genre unique values 
dfGenre = dfGenre.drop_duplicates().dropna().drop(dfGenre.index[10])

# split writers
# First_Writer = df.Writer.str.split(', ')


# explode the column
df = df.explode('Writer').reset_index(drop=True)


filterYear = df['Year'] >= 2010
for genre in dfGenre:
    filterGenre = df['GenreToCompare'].str.contains(pat = genre)
    dfTrend = dfTrend.append(df[['Original','Writer','Series or Movie','Year','GenreToCompare','IMDb Score']].where(filterYear & filterGenre, inplace = False).dropna().drop_duplicates().groupby(['Year','Original','Series or Movie','Writer']).agg(Films = ('Series or Movie', 'count')).assign(Genre = genre))

dfTrend = dfTrend.reset_index()
# first writer in a seperate column

#export to csv file

# dfTrend.sort_values(by=['IMDb Score'],ascending=False) 
dfTrend.sort_values(['Genre','Year','Original','Series or Movie','Writer'], ascending=[True,True,True,True,True], inplace=True)
dfTrend
#dfTrend.to_csv(r'D:\COMPUTER SCIENCE\2 SEMESTER\information visualization\project\trend_genre.csv', index = False, header=True)

,Year,Original,Series or Movie,Writer,Films,Genre
17646,2010.0,0.0,Movie,Aaron Lopresti,1,ACTION
17647,2010.0,0.0,Movie,Abhinav Kashyap,1,ACTION
17648,2010.0,0.0,Movie,Adam McKay,1,ACTION
17649,2010.0,0.0,Movie,Adi Hasak,1,ACTION
17650,2010.0,0.0,Movie,Alan Schoolcraft,1,ACTION
...,...,...,...,...,...,...
41775,2020.0,0.0,Movie,Nicole Belisle,1,WESTERN
41776,2020.0,0.0,Movie,Paul Greengrass,1,WESTERN
41777,2020.0,0.0,Movie,Paulette Jiles,1,WESTERN
41778,2020.0,0.0,Movie,Peter Carey,1,WESTERN


In [16]:
# split writers
df.Writer = df.Writer.str.split(', ')

# explode the column
df = df.explode('Writer').reset_index(drop=True)

In [17]:
df

,Title,Genre,Tags,Languages,Series or Movie,Hidden Gem Score,Country Availability,Runtime,Director,Writer,...,IMDb Votes,Image,Poster,TMDb Trailer,Trailer Site,TitleToCompare,GenreToCompare,BoxOfficeProfits,Original,Year
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,Thailand,< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,NaN,NaN,LETSFIGHTGHOST,"CRIME,DRAMA,FANTASY,HORROR,ROMANCE",2122065.0,0,2008.0
1,HOW TO BUILD A GIRL,Comedy,"Dramas,Comedies,Films Based on Books,British",English,Movie,7.0,Canada,1-2 hour,Coky Giedroyc,Caitlin Moran,...,2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ,YouTube,HOWTOBUILDAGIRL,COMEDY,70632.0,0,2020.0
2,Centigrade,"Drama, Thriller",Thrillers,English,Movie,6.4,Canada,1-2 hour,Brendan Walsh,Brendan Walsh,...,1720.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BODM2MD...,https://www.youtube.com/watch?v=0RvV7TNUlkQ,YouTube,CENTIGRADE,"DRAMA,THRILLER",16263.0,0,2020.0
3,Centigrade,"Drama, Thriller",Thrillers,English,Movie,6.4,Canada,1-2 hour,Brendan Walsh,Daley Nixon,...,1720.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BODM2MD...,https://www.youtube.com/watch?v=0RvV7TNUlkQ,YouTube,CENTIGRADE,"DRAMA,THRILLER",16263.0,0,2020.0
4,ANNE+,Drama,"TV Dramas,Romantic TV Dramas,Dutch TV Shows",Turkish,Series,7.7,"Belgium,Netherlands",< 30 minutes,NaN,NaN,...,1147.0,https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BNWRkMz...,NaN,NaN,ANNE+,DRAMA,NaN,0,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28458,K-POP Extreme Survival,NaN,"TV Dramas,TV Programmes,TV Comedies,Romantic T...",NaN,Series,NaN,"South Korea,Argentina,United Kingdom,Australia...",< 30 minutes,NaN,NaN,...,NaN,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN,K-POPEXTREMESURVIVAL,NAN,NaN,0,NaN
28459,DreamWorks Shreks Swamp Stories,NaN,"Animal Tales,Family Comedies,Family Adventures...",NaN,Series,NaN,"Russia,Hong Kong,Hungary,Australia,South Korea...",< 30 minutes,NaN,NaN,...,NaN,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN,DREAMWORKSSHREKSSWAMPSTORIES,NAN,NaN,0,NaN
28460,DreamWorks Happy Holidays from Madagascar,"Animation, Comedy, Family","TV Comedies,Kids TV,Animal Tales,TV Cartoons,T...",English,Series,8.4,"Belgium,Switzerland,United States,Germany,Unit...",< 30 minutes,NaN,NaN,...,71.0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,NaN,NaN,NaN,DREAMWORKSHAPPYHOLIDAYSFROMMADAGASCAR,"ANIMATION,COMEDY,FAMILY",NaN,0,NaN
28461,DreamWorks Holiday Classics,"Animation, Comedy, Family","TV Comedies,Kids TV,TV Cartoons,TV Programmes,...",English,Series,8.2,"Belgium,Switzerland,Germany,Argentina,United K...",< 30 minutes,NaN,NaN,...,82.0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN,DREAMWORKSHOLIDAYCLASSICS,"ANIMATION,COMEDY,FAMILY",NaN,0,NaN


In [18]:
First_Writer = df.Writer.str.split(', ')

In [19]:
First_Writer

0        [John Ajvide Lindqvist]
1                [Caitlin Moran]
2                [Brendan Walsh]
3                  [Daley Nixon]
4                            NaN
                  ...           
28458                        NaN
28459                        NaN
28460                        NaN
28461                        NaN
28462                        NaN
Name: Writer, Length: 28463, dtype: object